<a href="https://colab.research.google.com/github/satish860/see_food/blob/main/CIFAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load and normalize the CIFAR10 training and test datasets using torchvision

In [1]:
import torch
import torchvision
from torchvision import datasets,transforms

In [2]:
transform = transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

In [3]:
train_data = datasets.CIFAR10('/content/drive/MyDrive/cifar10',train=True,download=True,transform=transform)
test_data = datasets.CIFAR10('/content/drive/MyDrive/cifar10',train=False,download=True,transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [5]:
def imshow(img):
   img = img / 2 + 0.5     # unnormalize
   npimg = img.numpy()
   plt.imshow(np.transpose(npimg, (1, 2, 0)))
   plt.show()

In [6]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [7]:
device = get_default_device()
device

device(type='cuda')

In [8]:
train_Dataloader = DataLoader(train_data,batch_size=8,shuffle=True)
test_Dataloader = DataLoader(test_data,batch_size=8)

In [9]:
train_dl = DeviceDataLoader(train_Dataloader,device)
test_Dataloader = DeviceDataLoader(test_Dataloader,device)

# Define a Convolutional Neural Network

In [10]:
import torch.nn as NN
import torch.nn.functional as F

In [11]:
class cifar10Model(NN.Module):
  def __init__(self):
    super().__init__()
    self.network = NN.Sequential(
        NN.Conv2d(3,32,kernel_size=3,stride=1,padding=1),
        NN.ReLU(),
        NN.Conv2d(32,64,kernel_size=3,stride=1,padding=1),
        NN.ReLU(),
        NN.MaxPool2d(2,2),

        NN.Conv2d(64,128,kernel_size=3,stride=1,padding=1),
        NN.ReLU(),
        NN.Conv2d(128,128,kernel_size=3,stride=1,padding=1),
        NN.ReLU(),
        NN.MaxPool2d(2,2),
        
        NN.Conv2d(128,256,kernel_size=3,stride=1,padding=1),
        NN.ReLU(),
        NN.Conv2d(256,256,kernel_size=3,stride=1,padding=1),
        NN.ReLU(),
        NN.MaxPool2d(2,2),

        NN.Flatten(),
        NN.Linear(256*4*4,1024),
        NN.ReLU(),
        NN.Linear(1024,512),
        NN.ReLU(),
        NN.Linear(512,10)

    )

  def forward(self,xb):
    return self.network(xb)

In [12]:
model = to_device(cifar10Model(),device)
for images,label in train_dl:
  yb = model(images)
  break;

In [13]:
def accuracy(preds,labels):
  _,predicated = torch.max(preds,dim=1)
  return torch.tensor(torch.sum(predicated == labels).item() / len(preds))

In [14]:
accuracy(yb,label)

tensor(0.1250)

# Define a loss function

In [15]:
lossfn = F.cross_entropy

# Train the network on the training data

In [16]:
import torch.optim as optim

In [17]:
def train_loop(epochs, lr,model,train_dl,optim_fn=optim.Adam):
  optimizer = optim_fn(model.parameters(),lr)
  for epoch in range(epochs):
    train_loss = []
    train_acc = []
    for images,labels in train_dl:
      yb = model(images)
      loss = F.cross_entropy(yb,labels)
      train_loss.append(loss)
      acc = accuracy(yb,labels)
      train_acc.append(acc)
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()
    avg_loss = torch.stack(train_loss).mean()
    avg_acc = torch.stack(train_acc).mean()
    print("Epoch [{}], train_loss: {:.4f}, train_acc: {:.4f}".format(
            epoch, avg_loss,avg_acc))

In [18]:
cifarmodel = to_device(cifar10Model(),device)
train_loop(10,lr=0.001,model=cifarmodel,train_dl=train_dl)

Epoch [0], train_loss: 1.5696, train_acc: 0.4130
Epoch [1], train_loss: 1.0921, train_acc: 0.6091
Epoch [2], train_loss: 0.9109, train_acc: 0.6801
Epoch [3], train_loss: 0.8116, train_acc: 0.7174
Epoch [4], train_loss: 0.7480, train_acc: 0.7393
Epoch [5], train_loss: 0.6957, train_acc: 0.7587
Epoch [6], train_loss: 0.6560, train_acc: 0.7739
Epoch [7], train_loss: 0.6256, train_acc: 0.7846
Epoch [8], train_loss: 0.5884, train_acc: 0.7986
Epoch [9], train_loss: 0.5536, train_acc: 0.8122


In [20]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [19]:
def evaluate(model,test_dl):
  test_loss = []
  test_acc = []
  for images,label in test_dl:
    yb = model(images)
    loss = F.cross_entropy(yb,label)
    test_loss.append(loss)
    acc  = accuracy(yb,label)
    test_acc.append(acc)
    
  avg_loss = torch.stack(test_loss).mean()
  avg_acc = torch.stack(test_acc).mean()
  print("val_loss: {:.4f}, val_acc: {:.4f}".format(
             avg_loss,avg_acc))


In [23]:
torch.cuda.empty_cache()

In [20]:
evaluate(model,test_dl=test_Dataloader)

RuntimeError: ignored